In [66]:
import langchain
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import pandas as pd
import os
import json

In [8]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(temperature = 0.7,model = "gemini-pro")

In [10]:

print(llm.invoke("Who are you??").content)

I am Gemini, a large multi-modal model, trained by Google. I am designed to provide information and answer questions to the best of my abilities.


In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [54]:
template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
{response_json}

"""

In [55]:
prompt = PromptTemplate(template = template,input_variables = ["text","number","subject","tone","response_json"])
prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n{response_json}\n\n')

In [56]:
generate_chain = LLMChain(prompt = prompt,llm= llm,output_key = "quiz")


In [57]:
template2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [58]:
prompt_evaluation = PromptTemplate(input_variables = ["subject","quiz"],template = template2)
review_chain = LLMChain(prompt = prompt_evaluation,llm = llm,output_key = "review")

In [59]:
main_chain = SequentialChain(chains = [generate_chain,review_chain],
                             input_variables = ["text","number","subject","tone","response_json"],
                             output_variables = ["quiz","review"])

In [60]:
path = r"C:\Projects\MCQ-Generator\data.txt"
with open(path,"r") as file:
    text = file.read()
print(text[:200])
json.dumps(RESPONSE_JSON)

The Constitution of India is the supreme law of India.[2][3] The document lays down the framework that demarcates fundamental political code, structure, procedures, powers, and duties of government in


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [61]:
input_prompt = prompt.format(text=text,number=3,subject = "politics",tone="professional",response_json=RESPONSE_JSON)

In [62]:
response = llm.invoke(input_prompt)

In [64]:
print(response.content)

RESPONSE_JSON = {
    '1': {
        'mcq': 'Who was the chairman of the drafting committee for the Indian Constitution?',
        'options': {
            'a': 'Rajendra Prasad',
            'b': 'B. R. Ambedkar',
            'c': 'Jawaharlal Nehru',
            'd': 'Alladi Krishnaswamy Iyer'
        },
        'correct': 'b'
    },
    '2': {
        'mcq': 'Which of the following was NOT a source of inspiration for the Indian Constitution?',
        'options': {
            'a': 'Government of India Act 1858',
            'b': 'French Revolution',
            'c': 'Indian Independence Act 1947',
            'd': 'Government of India Act 1935'
        },
        'correct': 'b'
    },
    '3': {
        'mcq': 'When was the Constitution of India adopted by the Constituent Assembly?',
        'options': {
            'a': '26 November 1949',
            'b': '26 January 1950',
            'c': '15 August 1947',
            'd': '4 November 1947'
        },
        'correct': 'a'
    }

In [65]:
response_list = []

for key, value in RESPONSE_JSON.items():
    question = value['mcq']
    options = value['options']
    correct = value['correct']
    
    # Format each entry as a string
    formatted_entry = f"Question {key}: {question}\nOptions: {options}\nCorrect Answer: {correct}"
    response_list.append(formatted_entry)

print(response_list)

["Question 1: multiple choice question\nOptions: {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}\nCorrect Answer: correct answer", "Question 2: multiple choice question\nOptions: {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}\nCorrect Answer: correct answer", "Question 3: multiple choice question\nOptions: {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}\nCorrect Answer: correct answer"]


,0
0,Question 1: multiple choice question\nOptions:...
1,Question 2: multiple choice question\nOptions:...
2,Question 3: multiple choice question\nOptions:...
